In [1]:
import sys
import pandas as pd
import fnmatch
import os

repo_dir = os.path.join(os.getcwd(), "../../")
sanborn_raw_data_dir = os.path.join(repo_dir, 'data', 'sanborn_melanoma_2015')
data_dir = os.path.join(repo_dir, 'data', 'sanborn_melanoma_2015', 'cn_included_data')

# From Sanborn et. al. supplemental files
raw_data_fn = os.path.join(sanborn_raw_data_dir, "sanborn_melanoma_raw.xlsx")
PATIENTS = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Collect patient dataframes
raw_dfs = {}
for patient_id in PATIENTS:
    
    df = pd.read_excel(raw_data_fn, f"Patient {patient_id}")
    df['character_label'] = df.apply(lambda row: f"{row['Gene']}:{row['Chromosome']}:{row['Start position']}:{row['Alternate base']}", axis=1)
    # Drop rows where cn info is missing
    cn_cols = [x for x in df.columns if x.startswith("Copy number at base")]
    df = df.dropna(subset=cn_cols)
    raw_dfs[patient_id] = df
    print(patient_id, len(df))

/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


A 2109


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


B 135


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


...

/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


F 4696


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


G 767
H 3505


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Prepare inputs for pyclone clustering

In [2]:
pyclone_dir = os.path.join(data_dir, "pyclone_analysis")

def format_sample_name(name):
    return "_".join(name.lower().replace(",","").split())

def get_major_minor_cn(cn_str):
    items = cn_str.replace("(","").replace(")","").split(",")
    major_cn = int(float(items[0]))
    minor_cn = int(float(items[1]))
    return major_cn, minor_cn


patient_id_to_sample_names = {patient_id:[] for patient_id in PATIENTS}
pyclone_cols = ['mutation_id', 'ref_counts', 'var_counts', 'normal_cn', 'minor_cn', 'major_cn']

for patient_id,df in raw_dfs.items():
    print()
    print(patient_id)
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
    if not os.path.exists(os.path.join(pyclone_dir, patient_id)):
        os.makedirs(os.path.join(pyclone_dir, patient_id))
    data = []
    for sample in sample_names:
        cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
        subset = df[cols_to_keep]
        subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
        subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
        subset['ref_counts'] = subset.apply(lambda row: row[cols_to_keep[1]]-row[cols_to_keep[2]], axis=1)
        subset['var_counts'] = subset[cols_to_keep[2]] 
        subset['normal_cn'] = 2 # no sex chromosomes
        subset['mutation_id'] = subset[cols_to_keep[0]]
        subset = subset[pyclone_cols]
        print(subset['major_cn'])
        print(subset['minor_cn'])
        
        tsv_name = f"{patient_id}_{format_sample_name(sample)}.tsv"
        print(sample, tsv_name)
        subset.to_csv(os.path.join(pyclone_dir, patient_id, tsv_name), sep="\t")
        patient_id_to_sample_names[patient_id].append(format_sample_name(sample))
        
    print(sample_names)
    
    

/scratch/lsftmp/6206206.tmpdir/ipykernel_54549/984814212.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
/scratch/lsftmp/6206206.tmpdir/ipykernel_54549/984814212.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
/scratch/lsftmp/6206206.tmpdir/ipykernel_54549/984814212.py:29: SettingWithCopyWarning: 
A val


A
0       1
1       1
2       2
3       1
6       1
       ..
2724    1
2725    1
2726    1
2728    1
2729    1
Name: major_cn, Length: 2109, dtype: int64
0       1
1       1
2       1
3       1
6       1
       ..
2724    1
2725    1
2726    0
2728    1
2729    1
Name: minor_cn, Length: 2109, dtype: int64
primary, forehead A_primary_forehead.tsv
0       1
1       1
2       2
3       2
6       2
       ..
2724    1
2725    1
2726    1
2728    1
2729    1
Name: major_cn, Length: 2109, dtype: int64
0       1
1       1
2       1
3       1
6       1
       ..
2724    1
2725    1
2726    0
2728    1
2729    1
Name: minor_cn, Length: 2109, dtype: int64
parotid metastasis A_parotid_metastasis.tsv
0       1
1       1
2       1
3       1
6       1
       ..
2724    1
2725    1
2726    1
2728    1
2729    1
Name: major_cn, Length: 2109, dtype: int64
0       1
1       1
2       1
3       1
6       1
       ..
2724    1
2725    1
2726    0
2728    1
2729    1
Name: minor_cn, Length: 2109, dtype: 

0       2
1       2
2       2
3       2
4       2
       ..
4960    1
4964    1
4965    2
4966    1
4967    1
Name: major_cn, Length: 4696, dtype: int64
0       0
1       0
2       0
3       0
4       0
       ..
4960    1
4964    1
4965    1
4966    1
4967    1
Name: minor_cn, Length: 4696, dtype: int64
locoregional skin metastasis, left ear F_locoregional_skin_metastasis_left_ear.tsv
0       2
1       2
2       2
3       2
4       2
       ..
4960    1
4964    1
4965    2
4966    1
4967    1
Name: major_cn, Length: 4696, dtype: int64
0       0
1       0
2       0
3       0
4       0
       ..
4960    0
4964    1
4965    1
4966    1
4967    1
Name: minor_cn, Length: 4696, dtype: int64
distant skin metastasis, back F_distant_skin_metastasis_back.tsv
['primary, left ear', 'lymph node metastasis, left cervical node', 'locoregional skin metastasis, left ear', 'distant skin metastasis, back']

G
1      1
2      1
3      1
4      1
5      1
      ..
839    1
840    1
842    1
846    1
847  

### Setup pyclone commands for each patient

In [3]:
# Load tumor purities
purities = pd.read_csv(os.path.join(sanborn_raw_data_dir, "sanborn_melanoma_purities.csv"))
purities['Site'] = purities.apply(lambda row: format_sample_name(row['Site']), axis=1)
purities

,Patient,Site,Tumor Purity,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,A,primary_forehead,76,NaN,NaN,NaN
1,A,parotid_metastasis,21,NaN,NaN,NaN
2,A,locoregional_skin_metastasis_1_forehead,42,NaN,NaN,NaN
3,A,locoregional_skin_metastasis_2_angle_jaw,62,NaN,NaN,NaN
4,B,primary_mid-left_back,89,NaN,NaN,NaN
5,B,lymph_node_metastasis_left_axilla,23,NaN,NaN,NaN
6,B,locoregional_skin_metastasis_1_left_back,88,NaN,NaN,NaN
7,B,locoregional_skin_metastasis_2_left_axilla,51,NaN,NaN,NaN
8,C,primary_right_lower_calf,54,NaN,NaN,NaN
9,C,locoregional_skin_metastasis_1_right_calf,85,NaN,NaN,NaN


In [4]:
for patient_id,df in raw_dfs.items():
    cmd = [f"bsub -n 8 -W 60:00 -R rusage[mem=8] -o output_{patient_id}.log -e error_{patient_id}.log", "PyClone run_analysis_pipeline", "--in_files"]
    patient_dir = os.path.join(pyclone_dir, patient_id)
    
    for sample_name in patient_id_to_sample_names[patient_id]:
        cmd.append(os.path.join(patient_dir,  f"{patient_id}_{sample_name}.tsv"))
    cmd += ["--working_dir", patient_dir, "--tumour_contents"]
    
    # Add tumour cell proportions for each sample
    for sample_name in patient_id_to_sample_names[patient_id]:
        purity = float(purities[(purities['Patient']==patient_id) &(purities['Site']==sample_name)]['Tumor Purity'].item())/100
        cmd.append(str(purity))
    # Add sample names
    cmd.append("--samples")
    for sample_name in patient_id_to_sample_names[patient_id]:
        cmd.append(sample_name)
    cmd += ["--max_clusters", "30", "--min_cluster_size", "5", "--num_iters", "10000", "--burnin", "5000"]
    print(" ".join(cmd))
    

bsub -n 8 -W 60:00 -R rusage[mem=8] -o output_A.log -e error_A.log PyClone run_analysis_pipeline --in_files /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/data_preprocessing/../../data/sanborn_melanoma_2015/cn_included_data/pyclone_analysis/A/A_primary_forehead.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/data_preprocessing/../../data/sanborn_melanoma_2015/cn_included_data/pyclone_analysis/A/A_parotid_metastasis.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/data_preprocessing/../../data/sanborn_melanoma_2015/cn_included_data/pyclone_analysis/A/A_locoregional_skin_metastasis_1_forehead.tsv /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/data_preprocessing/../../data/sanborn_melanoma_2015/cn_included_data/pyclone_analysis/A/A_locoregional_skin_metastasis_2_angle_jaw.tsv --working_dir /lila/data/morrisq/divyak/projects/metient/metient/jupyter_notebooks/data_preprocessing/../../data/sanborn_me

## Prepare data for orchard tree inference


In [5]:
from metient.util import data_extraction_util as dutil
import json
orchard_dir = os.path.join(data_dir, "orchard_trees")

for patient_id,df in raw_dfs.items():
    
    patient_pyclone_dir = os.path.join(pyclone_dir, patient_id)
    cluster_id_to_mut_names, mutation_names = dutil.load_pyclone_clusters(os.path.join(patient_pyclone_dir, "tables", "loci.tsv"), min_mut_thres=5)
    cluster_df = pd.read_csv(os.path.join(patient_pyclone_dir, "tables", "loci.tsv"), sep="\t")
    for cid in cluster_id_to_mut_names:
        print(cid, len(cluster_id_to_mut_names[cid]))
    header = ["id", "name", "var_reads", "total_reads", "var_read_prob"]
    mut_name_to_mut_id = {}
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
        
    with open(os.path.join(orchard_dir, f"{patient_id}.ssm"), "w") as f:
        f.write("\t".join(header))
        f.write("\n")  
        for i, mut in enumerate(mutation_names):
            mut_name_to_mut_id[mut] = f"m{i}"
            row = [f"m{i}", mut]
            
            var_reads = []
            total_reads = []
            var_read_probs = []
        
            for sample in sample_names:
                
                cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
                subset = df[cols_to_keep]
                subset = subset[subset['character_label']==mut]
                subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
                subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)
            
                var = subset[cols_to_keep[2]].item()
                tot = subset[cols_to_keep[1]].item()
                major_cn = subset['major_cn'].item()
                minor_cn = subset['minor_cn'].item()
                formatted_sample_name = format_sample_name(sample)
                var_reads.append(str(var))
                total_reads.append(str(tot))
                p = float(purities[(purities['Patient']==patient_id) &(purities['Site']==formatted_sample_name)]['Tumor Purity'].item())/100
                var_read_prob = dutil.calc_var_read_prob(major_cn, minor_cn, p)
                var_read_probs.append(str(var_read_prob))

            row += [",".join(var_reads), ",".join(total_reads), ",".join(var_read_probs)]
            f.write("\t".join(row))
            f.write("\n")
    json_data = {"samples": sample_names, "clusters": [], "garbage": []}
    for x in range(0,len(cluster_id_to_mut_names)):
        json_data["clusters"].append([mut_name_to_mut_id[t] for t in cluster_id_to_mut_names[x]])

    with open(os.path.join(orchard_dir, f"{patient_id}.params.json"), 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False)

/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


CUDA GPU: False
0 1973
1 47
2 8
3 44
4 6
5 15
6 5
0 78
1 6
2 9
3 8
4 8
0 1040
1 42
2 175
3 94
4 27
5 39
6 5
0 923
1 21
2 21
3 5
0 9
1 8
2 9
3 62
4 19
5 8
6 9
7 7
0 3666
1 434
2 200
3 84
4 11
5 49
6 14
7 150
8 7
9 14
10 18
11 14
12 9
0 627
1 23
2 13
3 15
4 30
5 19
6 10
0 141
1 2076
2 1151
3 58
4 8
5 11
6 28
7 25
8 5


In [6]:
# Orchard commands
for patient_id in raw_dfs:
    
    cmd = f"bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_{patient_id}.log -e error_{patient_id}.log"
    cmd += f" python /data/morrisq/divyak/projects/orchard/bin/orchard -p {patient_id}.ssm {patient_id}.params.json {patient_id}.results.npz"
    print(cmd)
    

bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_A.log -e error_A.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p A.ssm A.params.json A.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_B.log -e error_B.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p B.ssm B.params.json B.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_C.log -e error_C.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p C.ssm C.params.json C.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_D.log -e error_D.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p D.ssm D.params.json D.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_E.log -e error_E.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p E.ssm E.params.json E.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8] -o output_F.log -e error_F.log python /data/morrisq/divyak/projects/orchard/bin/orchard -p F.ssm F.params.json F.results.npz
bsub -n 8 -W 10:00 -R rusage[mem=8

## Take PyClone generated clusters and create csvs with ref and var counts pooled by cluster

In [7]:
# Need a tsv for each patient with ['anatomical_site_index','anatomical_site_label', 'character_index', 'character_label', 'ref', 'var', 'var_read_prob', 'site_category', 'num_mutations']
import re
import numpy as np

from metient.util import data_extraction_util as dutil

final_cols = ['anatomical_site_index','anatomical_site_label', 'cluster_index', 'character_index','character_label', 'ref', 'var', 'var_read_prob', 'site_category']

for patient_id,df in raw_dfs.items():
    patient_pyclone_fn = os.path.join(pyclone_dir, patient_id, "tables", f"loci.tsv")
    mut_id_to_clstr_id, clstr_id_to_name, mutation_names = dutil.get_mut_to_cluster_map_from_pyclone_output(patient_pyclone_fn, 5)
    cols = list(df.columns)
    sample_names = [s.strip().lower() for s in cols[1:cols.index('Patient')]]
    print("patient", patient_id, len(clstr_id_to_name), "clusters", len(mutation_names), "mutations", len(sample_names), "samples")
    print(sample_names)
    
    data = []
    
    for midx, mut in enumerate(mutation_names):
        for sidx, sample in enumerate(sample_names):
            cols_to_keep = ['character_label', f"Total read depth ({sample})", f"Alternate base read depth ({sample})", f"Copy number at base ({sample})"]
            subset = df[cols_to_keep]
            subset = subset[subset['character_label']==mut]
            subset['major_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[0], axis=1)
            subset['minor_cn'] = subset.apply(lambda row: get_major_minor_cn(row[cols_to_keep[3]])[1], axis=1)

            var = subset[cols_to_keep[2]].item()
            tot = subset[cols_to_keep[1]].item()
            ref = tot - var
            major_cn = subset['major_cn'].item()
            minor_cn = subset['minor_cn'].item()
            formatted_sample_name = format_sample_name(sample)
            p = float(purities[(purities['Patient']==patient_id) &(purities['Site']==formatted_sample_name)]['Tumor Purity'].item())/100
            var_read_prob = dutil.calc_var_read_prob(major_cn, minor_cn, p)
            
            category = 'primary' if 'primary' in sample else 'metastasis'
            data.append([sidx, sample.capitalize(), mut_id_to_clstr_id[mut], midx, mut.split(":")[0], ref, var, var_read_prob, category])
            
    patient_df = pd.DataFrame(data, columns=final_cols)
    patient_df.to_csv(os.path.join(data_dir,"pyclone_clustered_tsvs", f"{patient_id}_SNVs.tsv"), sep="\t", index=False)


#     dutil.write_pooled_tsv_from_clusters(patient_df, mut_name_to_clstr_id, clstr_id_to_name, agg_rules, 
#                                          os.path.join(data_dir,"pyclone_clustered_tsvs"), f"patient{patient_id}", ";", ":")


patient A 7 clusters 2098 mutations 4 samples
['primary, forehead', 'parotid metastasis', 'locoregional skin metastasis 1, forehead', 'locoregional skin metastasis 2, angle jaw']
patient B 5 clusters 109 mutations 4 samples
['primary, mid-left back', 'lymph node metastasis, left axilla', 'locoregional skin metastasis 1, left back', 'locoregional skin metastasis 2, left axilla']
patient C 7 clusters 1422 mutations 3 samples
['primary, right lower calf', 'locoregional skin metastasis 1, right calf', 'locoregional skin metastasis 2, right mid-calf']
patient D 4 clusters 970 mutations 4 samples
['primary, right ankle', 'lymph node metastasis, right groin', 'locoregional skin metastasis 1, right ankle', 'locoregional skin metastasis 2, right leg']
patient E 8 clusters 131 mutations 5 samples
['primary, left heel', 'locoregional skin metastasis 1, left heel', 'locoregional skin metastasis 2, left heel', 'lymph node metastasis, left groin', 'locoregional skin metastasis 3, left heel']
patient